# **CUSTOM CLASSFIER FOR TEMPLES, BRIDGES, PETROL_PUMPS AND ATMS**

---



**IMPORTING LIBRARIES **

In [35]:
import os
import cv2
import keras
import pandas
import numpy as np
from random import shuffle
from keras import optimizers
from keras.models import Model
from keras import backend as K
from keras.optimizers import SGD
from keras.preprocessing import image
from keras.utils import np_utils as nu
from sklearn.model_selection import train_test_split
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import accuracy_score, confusion_matrix
from keras.layers import Dense, GlobalAveragePooling2D, Dropout,Input


TRAIN_DIR= 'drive//My Drive//Colab Notebooks//imcop'
IMG_SIZE= 299
LR= 1e-3
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**LABELER FUNCTION**

In [0]:
def label_img(img):
    word_label = img.split('-')[-2]
    if word_label == 'temple': return 3
    elif word_label == 'bridge': return 1
    elif word_label == 'petrol': return 2
    elif word_label == 'atm': return 0
    else: return 4

**INPUTING IMAGES LABELING AND SPLITTING IT**

In [0]:
training_xdata=[]
training_ydata=[]
for img in os.listdir(TRAIN_DIR):
    label=label_img(img)
    path= os.path.join(TRAIN_DIR, img)
    img= cv2.resize(cv2.imread(path),(IMG_SIZE,IMG_SIZE))
    training_xdata.append(np.array(img))
    training_ydata.append(np.array(label))

# np.save('training_xdata.npy', training_xdata)
# np.save('training_ydata.npy', training_ydata)

In [0]:
X=np.array(training_xdata)
Y=np.array(training_ydata)
X_train, X_test, Y_train_pre, Y_test_pre = train_test_split(X, Y, test_size = 0.15,random_state=4)
Y_train = nu.to_categorical(Y_train_pre)
Y_test = nu.to_categorical(Y_test_pre)

**BASE** **MODEL**

In [0]:
input_tensor = Input(shape=(299, 299, 3))  # this assumes K.image_data_format() == 'channels_last'
model = InceptionV3(input_tensor=input_tensor, weights='imagenet', include_top=False)



**ADDING OUR OWN LAYERS**

In [0]:
# add a global spatial average pooling layer
p = model.output
p = GlobalAveragePooling2D()(p)
# let's add a fully-connected layer
p = Dense(1024, activation='relu')(p)
# and a logistic layer --5 classes
predictions = Dense(5, activation='softmax')(p)
# this is the model we will train
myModel = Model(inputs=model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze  convolutional InceptionV3 layers
for layer in model.layers:
    layer.trainable = True
# for layer in myModel.layers[:133]:
#    layer.trainable = False
# for layer in myModel.layers[133:]:
#    layer.trainable = True

sgd = optimizers.SGD(lr=LR, momentum=0.9, nesterov=True)
myModel.compile(loss='categorical_crossentropy', optimizer=sgd, metrics= ['accuracy'])

**DATA AUGMENTATION**

In [0]:
datagen = ImageDataGenerator(
    featurewise_center=False,
    featurewise_std_normalization=False,
    rotation_range=10,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=False)
datagen.fit(X_train)

**TRAINING**


In [42]:

#myModel.fit(X_train, Y_train, batch_size=50,epochs=7, verbose=1)
myModel.fit_generator(datagen.flow(X_train, Y_train, batch_size=50),steps_per_epoch=len(X_train) / 50, epochs=30, verbose=1)

Epoch 1/30
16/15 [===============================] - 48s 3s/step - loss: 1.3230 - acc: 0.4699
Epoch 2/30
16/15 [===============================] - 27s 2s/step - loss: 0.7431 - acc: 0.8647
Epoch 3/30
16/15 [===============================] - 27s 2s/step - loss: 0.4121 - acc: 0.9261
Epoch 4/30
16/15 [===============================] - 27s 2s/step - loss: 0.2560 - acc: 0.9511
Epoch 5/30
16/15 [===============================] - 27s 2s/step - loss: 0.1816 - acc: 0.9511
Epoch 6/30
16/15 [===============================] - 26s 2s/step - loss: 0.1538 - acc: 0.9599
Epoch 7/30
16/15 [===============================] - 26s 2s/step - loss: 0.1067 - acc: 0.9862
Epoch 8/30
16/15 [===============================] - 27s 2s/step - loss: 0.0990 - acc: 0.9812
Epoch 9/30
16/15 [===============================] - 27s 2s/step - loss: 0.0708 - acc: 0.9900
Epoch 10/30
16/15 [===============================] - 27s 2s/step - loss: 0.0613 - acc: 0.9950
Epoch 11/30
16/15 [===============================] - 26s 2

**PREDICTION AND ACUURACY**

In [0]:
Y_pred=myModel.predict(X_test)


In [0]:

confusion_matrix(np.argmax(Y_pred),np.argmax(Y_test))#Ytest1 is preonhot 

In [64]:
scores = myModel.evaluate(X_test, Y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 98.53%


In [0]:
im= cv2.resize(cv2.imread('apparel.png'),(IMG_SIZE,IMG_SIZE))

In [68]:
im=im.reshape(1,299,299,3)
im.shape

(1, 299, 299, 3)

In [69]:
imd=np.array(im)
pp=myModel.predict(imd)
pp

array([[0.04518916, 0.00762212, 0.6553438 , 0.2475257 , 0.04431922]],
      dtype=float32)

In [0]:
import pickle
pickle.dump(myModel,open('cust_classifier.sav','wb'))